In [ ]:
import tensorflow
import matplotlib.pyplot as plt
import scipy
from keras.applications.vgg16 import VGG16

imagenetModel = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3)
                  )

imagenetModel.summary()

In [ ]:
imagenetModel.trainable = True
set_trainable = False
for layer in imagenetModel.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model = tensorflow.keras.models.Sequential()
model.add(imagenetModel)
model.add(tensorflow.keras.layers.Flatten())
model.add(tensorflow.keras.layers.Dense(256,activation='relu'))
model.add(tensorflow.keras.layers.Dense(4,activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer=tensorflow.keras.optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

model.summary()

In [ ]:
egitim_yolu = r'TrainPath'
gecerleme_yolu = r'ValidationPath'
test_yolu = r'TestPath'

train_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255, 
      rotation_range=40, 
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      brightness_range=[0.2,1.0],
      zoom_range=0.2,
      horizontal_flip = False,
      fill_mode='nearest'
      )

train_generator = train_datagen.flow_from_directory(
        egitim_yolu,
        target_size=(224, 224),
        batch_size=16,
        )

validation_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
        )

validation_generator = validation_datagen.flow_from_directory(
        gecerleme_yolu,
        target_size=(224, 224),
        batch_size=16,
        )

In [9]:
class MyThresholdCallback(tensorflow.keras.callbacks.Callback):
    def __init__(self, acc_limit,val_limit):
        super(MyThresholdCallback, self).__init__()
        self.acc_limit = acc_limit
        self.val_limit = val_limit
    def on_epoch_end(self, epoch, logs=None): 
        acc = logs["acc"]
        val_acc = logs["val_acc"]
        if val_acc >= self.val_limit :
            if acc >= self.acc_limit :
                self.model.stop_training = True

callbacks = MyThresholdCallback(acc_limit=0.85, val_limit=0.85)

In [ ]:
history = model.fit( 
      train_generator,
      steps_per_epoch=15,
      epochs=60,
      validation_data=validation_generator,
      validation_steps=1,
      callbacks=[callbacks]
      )



In [ ]:
model.save('ModelPath')

test_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
        )

test_generator = test_datagen.flow_from_directory(
       test_yolu,
        target_size=(224, 224),
        batch_size=16,
        )

In [ ]:
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_acc'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_acc', 'train_loss', 'validation_acc', 'validation_loss'], loc='upper left')
plt.show()

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)